Для проведения атаки установим модуль, содержащий в том числе и готовые реализации атак: Adversarial Robustness Toolbox
С документацией данного модуля вы можете ознакомиться дополнительно здесь: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/index.html


In [ ]:
!pip install adversarial-robustness-toolbox==1.10.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.5 MB/s 


Вы можете вручную загружать данные в рабочую область, а можете использовать свой смонтированный google-диск, для этого выполните следующий код:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Новый раздел

Атака будет проводиться на OpenSource проект VoxCeleb, представляющий собой нейронную сеть, определяющую принадлежат ли две записи голоса одному и тому же или различным говорящим знаменитостям. Исходный код проекта располагается здесь: https://github.com/clovaai/voxceleb_trainer
<br>Для удобства проведения атаки исходный код был частично модифицирован. Модифицированный для упрощенного проведения атаки код расположен в виде zip-архива, разархивируем его в рабочей области.

In [ ]:
!unzip './drive/MyDrive/Colab_Notebooks/Project_5/voice_attack.zip'

Archive:  ./drive/MyDrive/Colab_Notebooks/Project_5/voice_attack.zip
  inflating: DatasetLoader.py        
  inflating: loss/angleproto.py      
  inflating: main.py                 
  inflating: models/ResNetBlocks.py  
  inflating: models/ResNetSE34L.py   
  inflating: optimizer/adam.py       
  inflating: scheduler/steplr.py     
  inflating: SpeakerNet.py           
  inflating: tuneThreshold.py        
  inflating: utils.py                


In [ ]:
!unzip './drive/MyDrive/Colab_Notebooks/Project_5/part_voxceleb.zip'

Archive:  ./drive/MyDrive/Colab_Notebooks/Project_5/part_voxceleb.zip
   creating: part_voxceleb/id10270/
   creating: part_voxceleb/id10270/5r0dWxy17C8/
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00001.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00002.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00003.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00004.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00005.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00006.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00007.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00008.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00009.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00010.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00017.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00022.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00024.wav  
  inflating: part_voxceleb/id10270/5r0dWxy17C8/00026.wav  
  inflating: part_vo

Загрузим необходимые стандартные модули и модули проекта нейронной сети

In [ ]:
# импорт стандартных модулей
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from art.attacks.evasion import HopSkipJump
from art.estimators.classification import BlackBoxClassifierNeuralNetwork
from tensorflow.keras.utils import to_categorical
import soundfile

# импорт модулей VoxCeleb
from tuneThreshold import *
from SpeakerNet import *
from DatasetLoader import *

Прежде чем начать атаку выполним функции вычисления точности и статистических ошибок первого и второго рода на основе значений выходных предсказаний нейронной сети и истинных меток.

In [ ]:
def calc_accuracy(scores, labels):
    # scores - предсказания нейронной сети в категориальном виде
    # labels - истинные метки

    # вычисление предсказанных сетью меток
    predictions = np.argmax(scores, axis=1)

    labels = np.array(labels)

    # вычисление точности как отношения количества совпавших предсказанных 
    # и истинных меток к общему числу предсказаний
    accuracy = sum(predictions == labels) / len(predictions) * 100
    print('accuracy =', accuracy, '%')
    return accuracy


def calc_errors(scores, labels):
    # scores - предсказания нейронной сети в категориальном виде
    # labels - истинные метки

    # вычисление предсказанных сетью меток
    predictions = np.argmax(scores, axis=1)

    labels = np.array(labels)

    first = 0
    second = 0

    # нахождение всех несовпадающих с истинными предсказанных меток
    difference = np.where(predictions != labels)[0]

    # для каждой несовпавшей с истинной предсказанной меткой определяем
    # какого рода ошибка
    for diff in difference:
        # исходная посылка была истинной - результат ложно-отрицательный,
        # ошибка первого рода
        if labels[diff] == 1:
            first += 1
        # исходная посылка была ложной - результат ложно-положительный,
        # ошибка второго рода
        else:
            second += 1

    # вычисляем значение итоговых ошибок
    # итоговая ошибка первого рода - отношение количества ложно-отрицательных результатов
    # к общему числу истинных посылок
    first /= np.sum(labels == 1)
    # итоговая ошибка второго рода - отношение количества ложно-положительных результатов
    # к общему числу ложных посылок
    second /= np.sum(labels == 0)
    print('first = ', first * 100, '%; second = ', second * 100, '%')
    return first * 100, second * 100

Исходная нейронная сеть возвращает предсказания в виде одного значения схожести между двумя входными записями голоса для каждой тестовой пары.
Для проведения атаки, необходимо чтобы сеть возвращала результат предсказания в категориальном виде. Простыми словами категориальный вид предсказание - это представление предсказания нейронной сети не в виде одного числа, а в виде вектора значений, определяющих близость входных данных к каждому из классов. Это удобно при выполнении классификации данных, например, при распознавании изображений или рукописных подписей. В нашем случае также возможно выделить два класса: 0 - записи не принадлежат одному человеку и 1 - записи принадлежат одному человеку. Таким образом необходимо преобразовать одно значение, выдаваемое нейронной сетью к двум. Напишем функцию преобразования:

In [ ]:
def get_one_hot(scores, threshold):
    # scores - предсказания нейронной сети
    # threshold - порог принятия решения. Если значение предсказания выше порога
    # записи признаются принадлежащими одному человеку и не принадлежащими - в
    # противном случае
    result = []

    for score in scores:
        # преобразованное значение
        value = [0, 0]
        # степень уверенности сети в том, что записи не принадлежат одному человеку
        # определяется как отношение предсказания нейронной сети к порогу
        value[0] = score/threshold
        # обратная ситуация определяется обратным отношением
        value[1] = threshold/score
        result.append(value)

    return result

В рамках проекта VoxCeleb был собран достаточно большой датасет, в рамках данной работы для проведения нам нужен лишь небольшая часть данного датасета, представленная в материалах курса. Папка part_voxceleb содержит в себе записи участников в формате "идентификатор"/"ключ видео на youtube"/"файл записи в формате wav", а файл test_list.txt содержит список пар файлов с меткой, определяющей являются ли данные записи принадлежащими одному человеку: 0 - не являются, 1 - являются. Напишем функцию, загружающую файлы из part_voxceleb по списку указанному в test_list.txt, и возвращающая загруженные данные и соответствующие им метки.

In [ ]:
def create_attacked_dataset(files_list):
    # files_list - путь к файлу test_list.txt

    dataset = []
    all_labels = []

    with open(files_list) as f:
        lines = f.readlines()

    print('dataset loading...')
    for idx, line in enumerate(lines):
        data = line.split()

        # загрузка записей с помощью функции, определенной в проекте VoxCeleb
        audio1 = loadWAV(os.path.join(test_path, data[1]), 400, evalmode=True, num_eval=10)
        audio2 = loadWAV(os.path.join(test_path, data[2]), 400, evalmode=True, num_eval=10)
        
        voices = [audio1, audio2]
        dataset.append(voices)
        all_labels.append(int(data[0]))
    print('finished!')

    return np.array(dataset), np.array(all_labels)

Для проведения атаки необходимо, чтобы нейронная функция вызывалась через функцию, принимающую на вход данные, которые нужно классифицировать, и возвращающую результат в категориальном виде. 

In [ ]:
def get_prediction(dataset):
    # dataset - данные для классификации
    global trainer # контейнер с обученной моделью

    # рассчет предсказаний нейронной сети
    all_scores = []
    threshold = -1

    for couple in dataset:
        audio1 = torch.FloatTensor(couple[0])
        audio2 = torch.FloatTensor(couple[1])
        inp1 = audio1.cuda()
        inp2 = audio2.cuda()
        with torch.no_grad():
            ref1 = trainer.__model__(inp1).detach().cpu()
            ref2 = trainer.__model__(inp2).detach().cpu()

        ref_feat = ref1.cuda()
        com_feat = ref2.cuda()

        if trainer.__model__.module.__L__.test_normalize:
            ref_feat = F.normalize(ref_feat, p=2, dim=1)
            com_feat = F.normalize(com_feat, p=2, dim=1)

        dist = torch.cdist(ref_feat.reshape(10, -1), com_feat.reshape(10, -1)).detach().cpu().numpy()
        score = -1 * numpy.mean(dist)
        all_scores.append(score)

    # преобразование результатов в категориальный вид
    prediction = get_one_hot(all_scores, threshold)
    return prediction

Атака будет проводиться на предобученную модель baseline_lite_ap.model, относящуюся к классу ResNetSE34L. Исходное расположение сети: http://www.robots.ox.ac.uk/~joon/data/baseline_lite_ap.model
Итак, проведем атаку HotSkipJump. Опыт показывает, что данная атака проста в проведении и эффективна, кроме того для ее реализации нет необходимости иметь доступ непосредственно ко внутренней структуре сети, она проводится по методологии "черного ящика". Подробнее: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/attacks/evasion.html#hopskipjump-attack


In [ ]:
# отключим лишнее логирование действий атаки, чтобы не засорять вывод
import logging
logger = logging.getLogger('art.attacks.evasion.hop_skip_jump')
logger.disabled = True

# укажем расположение папки part_voxceleb и файла test_list.txt
test_list = './drive/MyDrive/Colab_Notebooks/Project_5/test_list.txt'
test_path = './part_voxceleb'

def main():
    global trainer

    # инициализируем и загружаем предобученную модель
    s = SpeakerNet('ResNetSE34L', 'adam', 'angleproto', nPerSpeaker=400, nOut=512)
    s = WrappedModel(s).cuda(0)
    trainer = ModelTrainer(s, 'adam', 'steplr', gpu=0, mixedprec=False, lr=0.001, lr_decay=0.95, weight_decay=0,
                           test_interval=10, max_epoch=500)
    trainer.loadParameters('./drive/MyDrive/Colab_Notebooks/Project_5/baseline_lite_ap.model')
    trainer.__model__.eval()

    # загружаем датасет для проведения атаки
    dataset, labels = create_attacked_dataset(test_list)

    # выведем форму загруженного датасета
    print(dataset.shape)
    """
    (100, 2, 10, 64240)
    100 - количество сравнений, для которых будет вычисляться схожесть
    2 - две аудиозаписи в каждом сравнении
    10 - каждая запись загружается в виде десяти отрезков
    64240 - каждый отрезок состоит из данного числа сэмплов
    """

    # вычислим значения предсказания для неатакованных данных
    pred = get_prediction(dataset)
    print(pred)

    # преобразуем предсказания к виду обычных меток
    print(np.argmax(pred, axis=1))

    # вычислим точность и значения ошибок
    calc_accuracy(pred, labels)
    calc_errors(pred, labels)

    # проводим атаку
    # создадим классификатор черного ящика, который позволяет проводить атаки
    # "черным ящиком" на основе функции, возвращающей предсказания нейронной сети
    classifier = BlackBoxClassifierNeuralNetwork(
        predict_fn=get_prediction,   # функция, возвращающая предсказания нейронной сети
        nb_classes=2,                # количество классов, в нашем случае их 2 - 0 - записи не принадлежат одному человеку и 1 - записи принадлежат одному человеку
        input_shape=(2, 10, 64240))  # форма загруженного датасета - все метрики, за исключением количества сравнений

    # атака будет проводиться только на одну аудиозапись из двух, таким образом
    # атакованная запись будет сравниваться с неатакованной. Суть атаки состоит 
    # в том, что несмотря на незначительные изменения исходной записи
    # нейронная сеть перестает адекватно воспринимать ее. Например, если две записи
    # принадлежат одному человеку, то после атаки на одну из них нейронная сеть не сможет
    # распознать их сходства и выдаст ложно-отрицательную метку, что приведет к общему
    # снижению точности распознавания в возрастанию ошибки первого рода

    # для того, что атаковывалась только одна запись из двух зададим маску
    # на значения маскированные нулем атака проводиться не будет
    # первая половина маски будет соответствовать первой аудиозаписи и будет состоять из 0
    #mask_zeros = np.zeros((10, 64240,))
    # вторая половина маски будет соответствовать второй аудиозаписи и будет состоять из 1
    #mask_ones = np.ones((10, 64240,))
    #mask = np.array([mask_zeros, mask_ones])

    # инициализируем атаку HopSkipJump
    attack = HopSkipJump(
        classifier, # ранее созданный классификатор черного ящика
        max_iter=1  # максимальное число итераций при генерации установлено в одну для того, чтобы генерация атакующих примеров не занимала слишком много времени
        )
    
    # генерируем атакующие примеры
    adv_dataset = attack.generate(x=dataset, y=labels)

    # вычислим значения предсказания для атакующих данных и вычислим точность и ошибки
    pred = get_prediction(adv_dataset)
    print(pred)
    print(np.argmax(pred, axis=1))
    calc_accuracy(pred, labels)
    calc_errors(pred, labels)
    """
    Ожидаемый результат
    """

    # сохраним один атакованный и не атакованный пример для сравнения данных до и 
    # после атаки
    soundfile.write('clear.wav',    dataset[0][1][0],     16000)
    soundfile.write('attacked.wav', adv_dataset[0][1][0], 16000)

    soundfile.write('attacked1.wav', adv_dataset[1][0][0],     16000)
    soundfile.write('attacked2.wav', adv_dataset[1][1][0], 16000)

if __name__ == '__main__':
    main()

Embedding size is 512, encoder SAP.
Initialised AngleProto
Initialised Adam optimizer
Initialised step LR scheduler
dataset loading...
finished!
(100, 2, 10, 64240)
[[0.8417819738388062, 1.1879560635394304], [1.1556305885314941, 0.8653284275477165], [0.7540985345840454, 1.3260866506677298], [1.1917729377746582, 0.8390860106852679], [0.7770070433616638, 1.2869896206777915], [1.159084677696228, 0.8627497362725721], [0.675621509552002, 1.4801186549893746], [1.0778743028640747, 0.9277519626758418], [0.7178911566734314, 1.3929688236219628], [1.2804243564605713, 0.7809910792108503], [0.7799628973007202, 1.282112269007641], [1.1005735397338867, 0.9086171563254147], [0.7380216717720032, 1.3549737605929406], [1.0102360248565674, 0.9898676897232795], [0.7478116750717163, 1.337235073127333], [1.2656681537628174, 0.7900965170270036], [0.7328441143035889, 1.364546675728283], [1.3109630346298218, 0.7627980145774066], [0.8080918788909912, 1.2374830463243605], [1.2252274751663208, 0.8161749718061564],

HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

[[0.8417819738388062, 1.1879560635394304], [0.8418691158294678, 1.1878330980401042], [0.7540985345840454, 1.3260866506677298], [0.8860176801681519, 1.1286456493850283], [0.7770070433616638, 1.2869896206777915], [0.7642419934272766, 1.3084860667175018], [0.675621509552002, 1.4801186549893746], [0.799858033657074, 1.2502218617819543], [0.7178911566734314, 1.3929688236219628], [0.9305142760276794, 1.0746745383305152], [0.7799628973007202, 1.282112269007641], [0.9333901405334473, 1.0713633630504005], [0.7380216717720032, 1.3549737605929406], [0.9753320813179016, 1.0252918151208215], [0.7478116750717163, 1.337235073127333], [0.8760610818862915, 1.141472918585596], [0.7328441143035889, 1.364546675728283], [0.8404855132102966, 1.1897885023388755], [0.8080918788909912, 1.2374830463243605], [0.8736571669578552, 1.144613743033873], [0.6433665752410889, 1.5543238310527243], [0.7707356810569763, 1.297461664975227], [0.8631208539009094, 1.1585863039694382], [0.9128173589706421, 1.0955094030285217],